#### Generate list of panels and get the forecasts using different methods

In [1]:
import numpy as np
import pandas as pd
from config import getClient
from forecastRF import forecastRF
from forecastSTL import forecastSTL
import warnings
warnings.filterwarnings('ignore')

DELIM = ","

In [2]:
dataloc = getClient("OutFront")

In [3]:
train = pd.read_csv(dataloc+"testData/train.csv")
test  = pd.read_csv(dataloc+"testData/test.csv")

# Convert their date format to a standard date
train["date"] = pd.to_datetime(train["date"])
test["date"]  = pd.to_datetime(test["date"])

In [4]:
outlier  = open(dataloc+"outliers.csv", "w")

In [5]:
def calcError(predictions, Y):
    errors = abs(predictions - Y)
    errors = round((errors/Y).mean(),2)
    return errors

In [6]:
def writePreds(panel, testDates, preds, testY):
    dates = testDates[0]
    hours = testDates[1]
    for x in range(len(dates)):
        rec = (panel+DELIM+str(dates[x])+DELIM+str(hours[x])+DELIM+str(preds["RF"][x])\
        +DELIM+str(preds["STL"][x])+DELIM+str(testY[x])+"\n")
        output.write(rec)

In [7]:
train = train.set_index("panel")
test  = test.set_index("panel")
grp   = train.groupby(level=0)

In [ ]:
nonFeatures = ["date", "hour", "population"]

panelList = []
datesList = []
hoursList = []

RFerrors    = []
STLerrors   = []
naiveerrors = []

RFpreds    = []
STLpreds   = []
actuals    = []

for panel, data in grp:
    tst = test.loc[panel]
    # These are simplistic forecasters: just use the Time Block
    meanMorning   = data.loc[data["block_morning"]  ==1]["population"].mean()
    meanAfternoon = data.loc[data["block_afternoon"]==1]["population"].mean()
    meanEvening   = data.loc[data["block_evening"]  ==1]["population"].mean()
    
    features = [x for x in data.columns if x not in nonFeatures]
    trainX = data[features]
    testX  = tst[features]
    trainX = np.array(trainX)
    testX  = np.array(testX)
    
    trainY = data["population"]
    testY  = tst["population"]
        
    d = {}                 # holds the data to pass to the forecasters
    d["trainX"] = trainX
    d["trainY"] = trainY
    d["testX"]  = testX
    d["testY"]  = testY
        
    predictions = forecastRF(d,200, 8, 16)
    errorRate   = calcError(predictions, testY)
    # A very high error rate usually means the data is messed up
    if errorRate < 4:
        RFerrors.append(errorRate)
        RFpreds.append(predictions)        
            
        predictions = forecastSTL(d)
        STLpreds.append(predictions)
        STLerrors.append(calcError(predictions, testY))
        
        # Get the predictions for a naive forecaster
        tst.loc[tst["block_morning"]  ==1, "naive"] = meanMorning
        tst.loc[tst["block_afternoon"]==1, "naive"] = meanAfternoon
        tst.loc[tst["block_evening"]  ==1, "naive"] = meanEvening
        naiveerrors.append(calcError(np.array(tst["naive"]), testY))
            
        panelList.append(panel)
        datesList.append(test["date"].reset_index(drop=True))
        hoursList.append(test["hour"].reset_index(drop=True))
        actuals.append(testY)
    else:
        rec = panel + "\n"
        outlier.write(rec)
outlier.close()

In [ ]:
# Summary has each panel and the errors associated with each algo
with open(dataloc+"summary.csv", "w") as output:
    hdr = ["panel","RF", "STL", "Naive"]
    hdr = ",".join(hdr) + "\n"
    output.write(hdr)
    errs = [list(a) for a in zip(panelList,RFerrors, STLerrors, naiveerrors)]

    for x in errs:
        rec = ",".join(map(str, x)) +"\n"
        output.write(rec)

In [ ]:
# "Predictions" has each panel and the errors associated with each algo
with open(dataloc+"predictions.csv", "w") as output:
    hdr = ["panel","date", "hour","actual","RF","STL"]
    hdr = ",".join(hdr) + "\n"
    output.write(hdr)
    for x, p in enumerate(panelList):
        
        preds = [list(a) for a in zip(datesList[x], hoursList[x], actuals[x], RFpreds[x], STLpreds[x])]
        for y in preds:
            rec = p +DELIM+DELIM.join(map(str, y)) +"\n"
            output.write(rec)